# Surface Reflection Correction 範本

這個notebook會：
1. 載入完整fiber raw data
2. 執行surface reflection correction
3. 將修正後的資料以parquet格式儲存（壓縮、省空間）

請根據實際需求調整檔案名稱、路徑與參數。

In [24]:
GAUGE_LENGTH = 10
SAMPLES = 1000
MULT = 4.0838
DAS_CNST = 116/8192*10**(-9)*SAMPLES/GAUGE_LENGTH

In [35]:
%load_ext autoreload
%autoreload 2
import os
import sys
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)
import pandas as pd
import numpy as np
from obspy import read
import tqdm
from scripts.surface_reflections_correction import correct_surface_reflection, load_das_profile
from scripts.picking_module import process_event

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
dasf = f"{RAW_DATA_PATH}/fiber/{event_date}.mseed"
start = start_time
end = end_time
channels = np.arange(1234,1405)

In [44]:
A = load_das_profile(dasf, start, end, channels)

100%|████████████████████████████████████| 1824/1824 [00:00<00:00, 14694.10it/s]


CoordinateValidationError: conflicting sizes for dimension 'channel': length 1824 on the data but length 171 on coordinate 'channel'

In [37]:
# 2. 設定檔案路徑
RAW_DATA_PATH = "/home/lulu/data/inci30"   
CORRECTED_DATA_PATH = '../data/fiber_corrected.parquet'  
event_date = "20231001-2221"
fiber_file = f"{RAW_DATA_PATH}/fiber/{event_date}.mseed"

picking = process_event(event_date, f"{RAW_DATA_PATH}/borehole", output_path=None)
start_time = picking['P_pick'] - 10
end_time = picking['S_pick'] + 5

In [42]:
# 3. 載入raw data
st = read(fiber_file, starttime=start_time, endtime=end_time)
A = np.zeros([len(st), st[0].stats.npts])
for i in tqdm.tqdm(np.arange(1234,1405)): 
    A[i, :] = st[i].data * DAS_CNST
# fiber_data = load_das_profile(fiber_file, )
# print(f'Raw data shape: {fiber_data.shape}')

  0%|                                                   | 0/171 [00:00<?, ?it/s]


IndexError: index 1234 is out of bounds for axis 0 with size 170

In [ ]:
# 4. 執行surface reflection correction
# 請根據你的函式實際參數調整這一行
corrected_fiber_data = correct_surface_reflection(fiber_data)
print(f'Corrected data shape: {corrected_fiber_data.shape}')

In [ ]:
# 5. 以parquet格式儲存（有壓縮，省空間）
corrected_fiber_data.to_parquet(CORRECTED_DATA_PATH, compression='snappy')
print(f'修正後資料已儲存為 {CORRECTED_DATA_PATH}')

## 後續分析建議
- 在 `run_analysis.ipynb` 只要載入 `fiber_corrected.parquet` 即可進行後續分析：

```python
import pandas as pd
df = pd.read_parquet('../data/fiber_corrected.parquet')
```

- 若修正參數有更動，重新執行此notebook即可產生新版資料。
- 若資料量極大，也可以考慮分批處理多個檔案。